# Move files exported by Equivital Qiosk

Check and clear 

Plot and review measurements from equivital sensors as output by the Equivital Manager app.
(Every program produces different formated files and it's a pain.)

In [1]:
import sys
import os
import shutil
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

## Definitions for data extraction

In [2]:
%load_ext autoreload
%autoreload 1
%aimport qex

In [12]:
%reload_ext autoreload

In [3]:
seperator = str('\\')
seperator

'\\'

# Scan for DATA files

All recordinds exported by Qiosk produce four CSV files and one SEM file. One of the CSVs is a DATA file that contains metadata and some essential signal quality checks on 15 s intervals.

We can ID instances of signal recordings from statistics taken out of the DATA file, so that gets extracted first

In [24]:
# location of the files when exported by QIOSK
path = "C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\"#Extracted SEM Data\\" #"../Equivital/Equivital Manager Wizard/Extracted SEM Data/"
#os.listdir(path)

df_datafiles = qex.qiosk_recordings(path,'Present',seperator)
df_datafiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VN102,3420426,230218,0,DATA-VN102-3420426-23021800.CSV,csv,70509,2023-02-18 13:25:57+00:00,2023-02-18 17:00:27+00:00,...,[VN102],3420426,VN102,NaN,0,100,0,1,Stationary,Upright
1,DATA,VN101,3420415,230218,0,DATA-VN101-3420415-23021800.CSV,csv,71160,2023-02-18 13:25:57+00:00,2023-02-18 17:00:27+00:00,...,[VN101],3420415,VN101,NaN,0,98,0,1,MovingSlowly,Upright
2,DATA,VN103,3420427,230218,0,DATA-VN103-3420427-23021800.CSV,csv,70869,2023-02-18 13:25:59+00:00,2023-02-18 17:00:29+00:00,...,[VN103],3420427,VN103,NaN,0,100,0,1,Stationary,Upright
3,DATA,VN104,3420456,230218,0,DATA-VN104-3420456-23021800.CSV,csv,70650,2023-02-18 13:26:01+00:00,2023-02-18 17:00:31+00:00,...,[VN104],3420456,VN104,NaN,0,100,0,1,Stationary,Prone
4,DATA,VN106,5022002,230218,0,DATA-VN106-5022002-23021800.CSV,csv,71093,2023-02-18 13:26:05+00:00,2023-02-18 17:00:35+00:00,...,[VN106],5022002,VN106,NaN,0,100,0,0,Stationary,Upright
5,DATA,VN105,3420755,230218,0,DATA-VN105-3420755-23021800.CSV,csv,70421,2023-02-18 13:26:05+00:00,2023-02-18 17:00:35+00:00,...,[VN105],3420755,VN105,NaN,0,98,0,1,Stationary,Upright
6,DATA,VN108,5022009,230218,0,DATA-VN108-5022009-23021800.CSV,csv,70636,2023-02-18 13:26:36+00:00,2023-02-18 17:00:51+00:00,...,[VN108],5022009,VN108,NaN,4,98,0,1,Stationary,Upright
7,DATA,VN107,5022006,230218,0,DATA-VN107-5022006-23021800.CSV,csv,70577,2023-02-18 13:26:36+00:00,2023-02-18 17:00:51+00:00,...,[VN107],5022006,VN107,NaN,0,97,0,1,Stationary,Upright
8,DATA,VN201,5022050,230218,0,DATA-VN201-5022050-23021800.CSV,csv,70558,2023-02-18 13:26:38+00:00,2023-02-18 17:00:53+00:00,...,[VN201],5022050,VN201,NaN,0,98,0,1,Stationary,Upright
9,DATA,VN202,5022051,230218,0,DATA-VN202-5022051-23021800.CSV,csv,70533,2023-02-18 13:26:41+00:00,2023-02-18 17:00:56+00:00,...,[VN202],5022051,VN202,NaN,0,99,0,1,Stationary,Upright


In [16]:
print('Number of DATA files: '+ str(len(df_datafiles)))
# key categories to structure of Qiosk output DATA files
print(df_datafiles['ID'].unique())
print(df_datafiles['DevName'].unique())
print(df_datafiles['Date'].unique())

Number of DATA files: 58
[5022154 5022198 5022202 3420415 3420426 3420427 3420456 3420755 5022002
 5022006 5022009 5022050 5022051 5022057 5022058 5022060 5022062 5022076
 5022073 5022077 5022098 5022100 5022101 5022110 5022104 5022117 5022126
 5022127 5022131 5022138 5022139 5022142 5022141 5023926 5022143 5022149
 5022151 5022155 5022156 5022164 5022163 5022161 5022166 5022173 5022184
 5022179 5022193 5022203 5022204 5022238 5022240 5022241 5022244 5022245
 5022256]
['LS410' 'BR604' 'BR605' 'VN101' 'VN102' 'VN103' 'VN104' 'VN105' 'VN106'
 'VN107' 'VN108' 'VN201' 'VN202' 'VN203' 'VN204' 'VN205' 'VN206' 'VN208'
 'VN207' 'VN209' 'VN210' 'VA301' 'VA302' 'VA304' 'VA303' 'VA305' 'VA306'
 'VA307' 'VA308' 'LS401' 'LS402' 'LS404' 'LS403' 'LS406' 'LS405' 'LS407'
 'LS408' 'WW501' 'WW502' 'WW505' 'WW504' 'WW503' 'WW506' 'WW507' 'BR602'
 'BR601' 'BR603' 'BR606' 'BR607' 'PC701' 'PC702' 'PC703' 'PC704' 'PC705'
 'PC707']
[230216 230217 230218]


## Define set of recordings to move
First specify a subset by either device number (singular) or day of recording event.

Next this subset can be refined by restricting recordings to a specific time of day: to have started before a particular time (when you know the recording was happening) and/or to have ended after a specific time (when you know the recording was happening.

All times are in UTC, which is an hour earlier than Central European Time, 2 hours earlier than Central European Summer Time.


In [11]:
# all the recordings that are in the directiory
s_files = df_datafiles

In [66]:
# DATA files from one device, by device ID numbr
devN = 3420456
s_files=df_datafiles.loc[df_datafiles['ID'] == devN,:] 

In [12]:
# DATA files from one device, by device ID numbr
devN = 'Bluetooth02'
s_files=df_datafiles.loc[df_datafiles['DevName'] == devN,:] 

In [21]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230217
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


In [9]:
# DATA files from recordings that lasted longer than N seconds
minDuration = 600  
s_files=df_datafiles.loc[df_datafiles['Duration'] > minDuration,:]


In [7]:
# DATA files from recordings that are smaller than N Bytes
maxfileSize = 500 
s_files=df_datafiles.loc[df_datafiles['FileSize'] < maxfileSize,:]


In [91]:
# DATA files from recordings that are smaller than N Bytes
maxdurSize = 50 
s_files=s_files.loc[s_files['Duration'] < maxdurSize,:]


Additionally select according to start and stop recording times.

In [82]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-14 12:02:00.00+0000')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']< t,:] 

In [68]:
# recordings ended after a certain time
t = pd.to_datetime('2023-01-28 00:00:00.00+0000')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecEnd']> t,:] 

In [22]:
# show subset of files
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
1,DATA,BR604,5022198,230217,0,DATA-BR604-5022198-23021700.CSV,csv,541,2023-02-17 13:41:46+00:00,2023-02-17 13:42:46+00:00,...,[BR604],5022198.0,BR604,NaN,10.0,97.0,1.0,1.0,Stationary,Side
2,DATA,BR605,5022202,230217,0,DATA-BR605-5022202-23021700.CSV,csv,537,2023-02-17 13:41:46+00:00,2023-02-17 13:42:46+00:00,...,[BR605],5022202.0,BR605,NaN,34.0,75.0,1.0,1.0,Stationary,Side


These recordings can be checked in View_Equivital_Qiosk_Export.

Now we move them

# Move selected files to suitable project folder

In [70]:
projectsFolder = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\' # where qiosk files are moved to after export
print(path) # where the qiosk files are initially saved by the Equivital Qiosk program

C:\Users\Public\Documents\Equivital\Equivital Manager Wizard\


In [71]:
# set project folder name
projectName = 'AlexanderStandStill'
#projectName = 'BatteryTests'
#projectName = 'SoloRecordings'
#projectName = 'Concert230128'
#projectName = 'LauraTestRecording'
projectPath = projectsFolder + projectName + '\\'

### Create folders for project to move files to
Check target location for data files to keep and to remove

In [72]:
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')

### Move CSV and SEM files to project folder
Move them to project folder and generate their own present recordings file.

for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath + 'CSV\\' + fileName
                os.rename(fi,out_f)
        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            os.rename(fi,out_f)
            
dfiles = qex.qiosk_recordings(projectPath,projectName)
dfiles

# Clear Recordings to ToBeDeleted

In [23]:
# if these are files to be discarded store the DATA file indexes to discardRecs
discardRecs = s_files.index
projTag = 'ToBeDiscarded'
discardPath = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\'

for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path,seperator) # outputs locations of csv and sem files
    
    for f in matched:
        fileName = f.split('\\')[-1]
        devName = row['DevName']
        if f.lower().endswith('csv'):
            out_f = discardPath + 'CSV\\' + fileName
            shutil.move(f,out_f)
        if f.lower().endswith('sem'):
            if not os.path.isdir(discardPath + 'SEM\\' + devName):
                os.mkdir(discardPath + 'SEM\\' + devName)
            out_f = discardPath + 'SEM\\' + devName + '\\' + fileName
            shutil.move(f,out_f)
            


In [26]:
dfiles = qex.qiosk_recordings(discardPath,projTag,seperator)
print(len(dfiles))

836


# if Files need to be moved between project folders

Set the origin project name and target project name, collect the existing files at that location, chose the subset of recordings to move or copy, move or copy them, generate recording list in each location

In [40]:
# incase files need to be shifted from a project folder
projectName = 'SoloRecordings'
projectPath = projectsFolder + projectName + '\\'
projectPath1 = projectPath
projectName1 = projectName

s_files = qex.qiosk_recordings(projectPath1,projectName1)
s_files 

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Test1,3420415,230120,7,DATA-Test1-3420415-23012007.CSV,csv,858075,2023-01-20 22:00:08+00:00,2023-01-21 13:31:13+00:00,...,[Test1],3420415.0,Test1,NaN,61,99,0,0,MovingSlowly,Side
1,DATA,Pilot_1,3420755,230201,102,DATA-Pilot_1-3420755-23020100_1.CSV,csv,62494,2023-02-01 17:14:56+00:00,2023-02-01 20:30:11+00:00,...,[Pilot_1],3420755.0,Pilot_1,NaN,78,98,0,0,Stationary,Upright
2,DATA,Bluetooth_Tester_1,3420415,230203,0,DATA-Bluetooth_Tester_1-3420415-23020300.CSV,csv,196,2023-02-03 10:10:03+00:00,2023-02-03 10:10:03+00:00,...,[Bluetooth_Tester_1],3420415.0,Bluetooth_Tester_1,NaN,30,97,1,1,Stationary,Side


In [41]:
# get subset of s_files as you see fit
s_files = s_files.loc[2:2,:].copy()
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
2,DATA,Bluetooth_Tester_1,3420415,230203,0,DATA-Bluetooth_Tester_1-3420415-23020300.CSV,csv,196,2023-02-03 10:10:03+00:00,2023-02-03 10:10:03+00:00,...,[Bluetooth_Tester_1],3420415.0,Bluetooth_Tester_1,NaN,30,97,1,1,Stationary,Side


In [42]:
# project these files should be moved to
projectName = 'ToBeDeleted'
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')
    
projectPath = projectsFolder + projectName + '\\'
projectPath2 = projectPath
projectName2 = projectName

In [43]:
print(row['FullLoc'])
matched = qex.matched_files(dataFile,projectPath1)

C:\Users\fourMs lab\Documents\Equivital\ProjectData\SoloRecordings\CSV\DATA-Bluetooth_Tester_1-3420415-23020300.CSV


In [54]:
# move s_files
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,projectPath1) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath2 + 'CSV\\' + fileName
                shutil.move(fi,out_f)
        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectPath2 + 'SEM\\' + devName):
                os.mkdir(projectPath2 + 'SEM\\' + devName)
            out_f = projectPath2 + 'SEM\\' + devName  + '\\' + fileName
            shutil.move(fi,out_f)

In [49]:
# COPY s_files
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,projectPath1) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath2 + 'CSV\\' + fileName
                #os.system('cp ' + fi + ' ' + out_f)
                shutil.copy2(fi,out_f) # complete target filename given

        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectPath2 + 'SEM\\' + devName):
                os.mkdir(projectPath2 + 'SEM\\' + devName)
            out_f = projectPath2 + 'SEM\\' + devName  + '\\' + fileName
            shutil.copy2(fi,out_f) # complete target filename given


In [50]:
fi

'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\SoloRecordings\\SEM\\Bluetooth_Tester_1\\23020300.SEM'

In [51]:
print(projectPath2)
out_f

C:\Users\fourMs lab\Documents\Equivital\ProjectData\ToBeDeleted\


'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\SEM\\Bluetooth_Tester_1\\23020300.SEM'

In [55]:
dfiles = qex.qiosk_recordings(projectPath1,projectName1)
print(projectPath1+projectName1)
print(dfiles)
dfiles = qex.qiosk_recordings(projectPath2,projectName2)
print(projectPath2+projectName2)
print(dfiles)

C:\Users\fourMs lab\Documents\Equivital\ProjectData\SoloRecordings\SoloRecordings
  Signal  DevName       ID    Date  Session  \
0   DATA    Test1  3420415  230120        7   
1   DATA  Pilot_1  3420755  230201      102   

                              FileName FileType  FileSize  \
0      DATA-Test1-3420415-23012007.CSV      csv    858075   
1  DATA-Pilot_1-3420755-23020100_1.CSV      csv     62494   

                   RecStart                    RecEnd  ...  SubjectNames  \
0 2023-01-20 22:00:08+00:00 2023-01-21 13:31:13+00:00  ...       [Test1]   
1 2023-02-01 17:14:56+00:00 2023-02-01 20:30:11+00:00  ...     [Pilot_1]   

   SENSOR ID  SUBJECT ID  SUBJECT AGE HR(BPM) HRC(%)  BELT OFF LEAD OFF  \
0  3420415.0       Test1          NaN      61     99         0        0   
1  3420755.0     Pilot_1          NaN      78     98         0        0   

         MOTION  BODY POSITION  
0  MovingSlowly           Side  
1    Stationary        Upright  

[2 rows x 25 columns]
C:\Users\fourMs